2020-01-24: something was wrong with the skw routine in tasks_mpi. Turned out that we are 
still calculating the conditional spectra one-by-one for each channel.
Some indentation was missing

In [4]:
%load_ext autoreload
%autoreload 2

In [73]:
import sys
sys.path.append("/global/homes/r/rkube/repos/delta")
import numpy as np
import json
import more_itertools

In [74]:
#from analysis.tasks_mpi import bicoherence
from analysis.task_fft import task_fft_scipy

In [75]:
from analysis.ecei_helper import channel_position

In [76]:
from analysis.channels import channel, channel_range, channel_pair

In [77]:
with open("../configs/test_skw.json") as df:
    cfg = json.load(df)

In [78]:
# Create an fft object and generate fft data as in processor_mpi
cfg["fft_params"]["fsample"] = cfg["ECEI_cfg"]["SampleRate"] * 1e3
my_fft = task_fft_scipy(10_000, cfg["fft_params"], normalize=True, detrend=True)

fft_params = my_fft.get_fft_params()

In [79]:
data = np.random.uniform(0.0, 1.0, [192, 10_000])

In [80]:
fft_data = my_fft.do_fft_local(data)

In [81]:
fft_data.shape

(192, 512, 38)

In [82]:
# Create a dummy channel pair list

In [83]:
chrg_ref = channel_range.from_str(cfg["task_list"][0]["ref_channels"])
chrg_cmp = channel_range.from_str(cfg["task_list"][0]["cmp_channels"])

channel_pairs = [channel_pair(cr, cx) for cr in chrg_ref for cx in chrg_cmp]

chpair_list = list([i[0] for i in more_itertools.distinct_combinations(channel_pairs, 1)])


#chpair_list = [channel_pair(ch1, ch2) for ch1, ch2 in zip(chrg_ref, chrg_cmp)]

In [84]:
print(chpair_list[2])

channel_pair: (ch1=L0101, ch2=L0103)


In [85]:
def skw(fft_data, ch_it, fft_params, ecei_config, info_dict, kstep=0.01): 
    """
    Calculates the conditional spectrum S(k,w).

    Input:
    ======
    fft_data: dask_array, float: Contains the fourier-transformed data. dim0: channel, dim1: Fourier Coefficients
    ch0: channel, first channel
    ch1: channel, second channel
    fft_params: dictionary, parameters for fft
    ecei_config: dictionary, configuration of ecei diagnostic


    Returns:
    ========
    bicoherence, float.
    """

    from analysis.ecei_helper import channel_position

    res_list = []
    for ch_pair in ch_it:

        ch1 = ch_pair.ch1
        ch2 = ch_pair.ch2
        ch1_idx, ch2_idx = ch1.idx(), ch2.idx()
        print("Calculating skw for channels {0}x{1}".format(ch1, ch2))

        nfft = int(fft_params["nfft"])

        if(ch1_idx == ch2_idx):
            # We can't calculate the cross-conditional spectrum for ch0==ch1
            res_list.append(None)
            continue

        XX = np.fft.fftshift(fft_data[ch1_idx, :, :], axes=0).T
        YY = np.fft.fftshift(fft_data[ch2_idx, :, :], axes=0).T

        bins, _ = XX.shape
        win_factor = fft_params["win_factor"]

        cpos_ref = channel_position(ch1, ecei_config)
        cpos_cmp = channel_position(ch2, ecei_config)

        # Calculate distance between channels
        dist = np.sqrt( (cpos_ref[0] - cpos_cmp[0])**2.0 + (cpos_ref[1] - cpos_cmp[1])**2.0)
        dmin = dist * 1e2

        kax = np.arange(-np.pi / dmin, np.pi / dmin, kstep)

        nkax = kax.size


        # value dimension
        Pxx = np.zeros((bins, nfft), dtype=np.complex_)
        Pyy = np.zeros((bins, nfft), dtype=np.complex_)
        Kxy = np.zeros((bins, nfft), dtype=np.complex_)
        val = np.zeros((nkax, nfft), dtype=np.complex_)

        sklw = np.zeros((nkax, nfft), dtype=np.complex_)
        K = np.zeros(nfft, dtype=np.complex_)
        sigK = np.zeros(nfft, dtype=np.complex_)

        #logging.info(f"nkax = {nkax:d}, nfft = {nfft:d}, dmin = {dmin:f}")

        # calculate auto power and cross phase (wavenumber)
        for b in range(bins):
            X = XX[b, :]
            Y = YY[b, :]

            Pxx[b,:] = X*np.matrix.conjugate(X) / win_factor
            Pyy[b,:] = Y*np.matrix.conjugate(Y) / win_factor
            Pxy = X*np.matrix.conjugate(Y)
            Kxy[b,:] = np.arctan2(Pxy.imag, Pxy.real).real / (dist * 100) # [cm^-1]
                                                            
            # calculate SKw
            for w in range(nfft):
                idx = (Kxy[b,w] - kstep * 0.5 < kax) * (kax < Kxy[b,w] + kstep * 0.5)
                val[:,w] = val[:,w] + (1.0 / bins * (Pxx[b,w] + Pyy[b,w]) * 0.5) * idx

        # calculate moments
        sklw = val / np.tile(val.sum(axis=0), (nkax, 1))
        K[:] = np.sum(np.transpose(np.tile(kax, (nfft, 1))) * sklw, axis=0)
        for w in range(nfft):
            sigK[w] = np.sqrt(np.sum( (kax - K[w])**2 * sklw[:,w] ))

        val = val.mean(axis=0).real
        K = np.mean(K, axis=0)
        sigK = np.mean(sigK, axis=0)
        pdata = np.log10(val + 1e-10)

        res_list.append(pdata)

    print("Finished skw analysis. len(res_list) = ", len(res_list))

    return(res_list, info_dict)

In [86]:
print(chpair_list[1].ch1, chpair_list[1].ch2)

L0101 L0102


In [87]:
for ch_pair in chpair_list:
    ch1 = ch_pair.ch1
    ch2 = ch_pair.ch2
    
    cpos_ref = channel_position(ch1, cfg['ECEI_cfg'])
    cpos_cmp = channel_position(ch2, cfg['ECEI_cfg'])
    dist = np.sqrt( (cpos_ref[0] - cpos_cmp[0])**2.0 + (cpos_ref[1] - cpos_cmp[1])**2.0)

    print(ch1, cpos_ref, ch2, cpos_cmp, dist)

L0101 (1.3813407218347238, -0.20742868943129145, -0.009121291139657118) L0101 (1.3813407218347238, -0.20742868943129145, -0.009121291139657118) 0.0
L0101 (1.3813407218347238, -0.20742868943129145, -0.009121291139657118) L0102 (1.3813407218347238, -0.18939141208944, -0.008328135388382587) 0.018037277341851454
L0101 (1.3813407218347238, -0.20742868943129145, -0.009121291139657118) L0103 (1.3813407218347238, -0.17135413474758857, -0.0075349796371080545) 0.03607455468370288
L0101 (1.3813407218347238, -0.20742868943129145, -0.009121291139657118) L0104 (1.3813407218347238, -0.15331685740573717, -0.006741823885833522) 0.05411183202555428
L0102 (1.3813407218347238, -0.18939141208944, -0.008328135388382587) L0102 (1.3813407218347238, -0.18939141208944, -0.008328135388382587) 0.0
L0102 (1.3813407218347238, -0.18939141208944, -0.008328135388382587) L0103 (1.3813407218347238, -0.17135413474758857, -0.0075349796371080545) 0.018037277341851427
L0102 (1.3813407218347238, -0.18939141208944, -0.0083281

In [88]:
res = skw(fft_data, chpair_list[0:2], fft_params, cfg['ECEI_cfg'], None)

Calculating skw for channels L0101xL0101
Calculating skw for channels L0101xL0102
Finished skw analysis. len(res_list) =  2


SyntaxError: invalid syntax (<ipython-input-72-664caa0ae96d>, line 1)

In [48]:
GG, _ = coherence(fft_data, chpair_list, fft_params, cfg['ECEI_cfg']);

In [56]:
np.fabs(GG.real)

array([[0.05263158, 0.1705434 , 0.03399223, 0.09730163, 0.1151428 ,
        0.04514039, 0.02582669, 0.04684854, 0.111333  , 0.00809159,
        0.0078383 , 0.1420049 , 0.14719808, 0.16388938, 0.12367904,
        0.1120856 , 0.03497395, 0.17878891, 0.04959764, 0.2273268 ,
        0.13756835, 0.00813338, 0.21021674, 0.23607948, 0.0524827 ,
        0.07427034, 0.02956471, 0.12235664, 0.00750519, 0.04539071,
        0.17916188, 0.15265389, 0.19430126, 0.23531223, 0.15693133,
        0.18826967, 0.0451555 , 0.05531797, 0.10789615, 0.05386624,
        0.03803497, 0.16346597, 0.03027807, 0.0820435 , 0.09863577,
        0.19532314, 0.14523585, 0.12383789, 0.38949928, 0.03035503,
        0.15306313, 0.12374617, 0.20714476, 0.11333828, 0.29520554,
        0.19653101, 0.14381102, 0.18917168, 0.15711477, 0.05406886,
        0.06412951, 0.0140791 , 0.06638046, 0.09204417, 0.15601659,
        0.09066992, 0.17675651, 0.0538422 , 0.02239427, 0.11734348,
        0.01167855, 0.04430294, 0.02274227, 0.06

In [52]:
GG.dtype

dtype('complex128')

In [54]:
?np.abs

Call signature:  np.abs(*args, **kwargs)
Type:            ufunc
String form:     <ufunc 'absolute'>
File:            ~/.conda/envs/delta/lib/python3.7/site-packages/numpy/__init__.py
Docstring:      
absolute(x, /, out=None, *, where=True, casting='same_kind', order='K', dtype=None, subok=True[, signature, extobj])

Calculate the absolute value element-wise.

``np.abs`` is a shorthand for this function.

Parameters
----------
x : array_like
    Input array.
out : ndarray, None, or tuple of ndarray and None, optional
    A location into which the result is stored. If provided, it must have
    a shape that the inputs broadcast to. If not provided or `None`,
    a freshly-allocated array is returned. A tuple (possible only as a
    keyword argument) must have length equal to the number of outputs.
where : array_like, optional
    This condition is broadcast over the input. At locations where the
    condition is True, the `out` array will be set to the ufunc result.
    Elsewhere, the `o